In this notebook we will generate and save the data frames that we will use in visualization, using the clean ones done in the previous steps

In [10]:
import pandas as pd
sys.path.append('../')
import src.limpieza as lmp

In [7]:
df_civitatis_clean = pd.read_pickle(r'C:\Users\mituc\Ironhack\Curso\IronLabs\Proyecto-1\data\civitatis_clean.pkl')
df_turistas_admon = pd.read_pickle(r'C:\Users\mituc\Ironhack\Curso\IronLabs\Proyecto-1\data\Barcelona_guiris_esquilados.pkl')

# Civitatis

Total number of tourist grouped by month

In [ ]:
lmp.total_civitatis_tourists_month(df_civitatis_clean)

# Spanish goverment

As civitatis started its activity for the tour that we are studying in June 2017, we'll select that period from the data frame we had generated using the data given by the goverment.


In [ ]:
lmp.admon_cividates(df_turistas_admon)